# 1 - Problem statement
It is January 2022.

You just started your new position: you've joined the Applied Machine Learning team of an Italian utility. 

You barely had the time to meet your new colleagues and you already have your first task. Up until now, the company relied on an external service to forecast the power demand of its customers. It is now time to internalize. 

Power demand forecasting is a critical task for every utility: power storage is not cheap nor widely available, so the balance of the grid must be guaranteed at any time. The production must match the demand. In Italy, this is ensured by the free power market, where utilities can trade power production, and by *ad-hoc* actions performed by the Transmission System Operator (TSO).

To safeguard the smooth operation of the system, utilities must produce day-ahead hourly forecasts of the power demand of their customers, and they get financial penalties for errors. 

<div class="alert alert-block alert-warning">
<b>Simplification.</b> 

We will consider daily forecasting of the whole Italian load, instead of hourly forecast of the demand from the customers of a specific company.
Moreover, instead of day-ahead forecasting we will focus on the long term.
</div>

# 2 - Data retrieval
Historical power load data can be retrieved from the [ENTSO-E portal](https://www.entsoe.eu/data/power-stats/), while newer series are avaiable on the websites of the national TSOs. In Italy, the TSO is Terna, and it [publishes power load and its own forecast](https://www.terna.it/en/electric-system/transparency-report/total-load).

After a bit of data wrangling you manage to get all historical data into a single csv file.

In [ ]:
import gdown

url = "https://drive.google.com/drive/folders/1esN41Xw_XfKgNh6cxj9YVXJCAtQhaAP_?usp=sharing"
gdown.download_folder(url, quiet=False)

In [ ]:
import pandas as pd

In [ ]:
raw_df = pd.read_csv("data/2006_2021_data.csv", index_col=0, parse_dates=True)
raw_df

Since you care about daily data, you sum the load on the same day.

In [ ]:
resampled_df = raw_df.resample('D').sum()

In [ ]:
load_series = resampled_df.Load
load_series.head()

# 3 - Exploratory Data Analysis
You did your homework: the features of the power load series are subject of an extensive literature, so you already know what to look for.

You start by plotting the series with an interactive view for a quick visual inspection.

In [ ]:
import plotly.express as px

px.line(load_series)

Some features are immediately visible:
- the trend is decreasing
- there are weekly and yearly seasonalities
- summer is the period of highest consumption, due to air conditioning
- there are some drops in correspondence with holidays, which reduce the demand from industrial plants
- the size of the peak changes from year to year, possibly due to weather conditions
- there is a pretty big drop in demand around march 2020

You further explore the trend with a moving average filter, in order to remove high-frequency oscillations.

In [ ]:
px.line(load_series.rolling(365).mean())

You then have a look at the **autocorrelation** function, a common statistical analysis which is very useful to detect and quantify seasonality patterns.

The autocorrelation of a real-valued stochastic process $X(t)$ is a function of two time variables, defined as 
$$
R_{XX}(t_1, t_2) = \mathbb{E}\left[X_{t_1}X_{t_2} \right],
$$
and it measures the Pearson correlation between values of the $X$ process taken at those two times.

In practice, usually we estimate the autocorrelation of a single realization of a process, i.e. a time series of the form $\left\{x(t)\right\}_{t=1}^n$, as a function of the difference $k = t_1 - t_2$, often called **lag**:
$$
\hat{R}(k)=\frac{1}{(n-k) \sigma^2} \sum_{t=1}^{n-k}\left(x(t)-\mu\right)\left(x(t+k)-\mu\right)
$$
where $\mu$ and $\sigma$ are also estimated from the time series.

A high autocorrelation value for a given lag $k$ means that the series is highly correlated with a version of itself that has been shifted $k$ steps, which can be intuitively interpreted as the presence of a strong repetitive component having frequency $1/k$.

In [ ]:
plot_acf(load_series, lags=370);

Let's focus on the first lags to better appreciate the highest autocorrelation values.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

plot_acf(load_series, lags=20);

The ACF shows a weekly seasonality, as well as a longer periodicity, which we may assume to be yearly. This fact can be confirmed by plotting the periodogram, an estimator of the spectral power density of the time series. The analysis is here omitted.

Finally, you plot the power demand juxtaposed year-over-year, a view that is interesting

In [ ]:
import plotly.graph_objects as go

year_over_year_df = pd.DataFrame({
    'load': load_series,
    'day_in_year': load_series.index.dayofyear,
    'day_in_week': load_series.index.dayofweek, # Monday = 0, Sunday = 6
    'year': load_series.index.year
})
fig = go.Figure(
    layout=dict(
        title='Adjusted day in year - if each year started on Monday'
    )
)
palette = iter(px.colors.sequential.Viridis[2:])
for year, year_df in year_over_year_df.groupby('year'):
  if year >= 2015:
    year_df.day_in_year += year_df.day_in_week.iloc[0]
    fig.add_trace(go.Scatter(x=year_df.day_in_year, y=year_df.load, name=year, marker_color=next(palette)))
fig.show()

# 4 - Modeling

The modeling phase involves choosing a parameterized family of functions which we hope approximate well the target series and then run some algorithm that finds the best candidate within the family.

Usually, one tries different models and then compares them to pick the best one; here we will just showcase a single good candidate model.

In practice, after choosing the functional form of the model we will apply it on our data, which involves:
- splitting the dataset at a certain date into two chunks;
- using the first chunk to train the model, i.e. determining the parameters that provide the best fit to it;
- using the second chunk to test how good the fit is when applied to unseen data.

This subdivision is fundamental if we want to check that our model can generalize, i.e. perform well on new data that comes from the same generating process but exhibits different behaviors due to the stochastic nature of the process itself.

## The model: Fourier regression

Many methods for long-term forecasting aim at identifying and reconstructing the trend and seasonal structure of the series.
When sesonal components are important, as in this case, a common approach is [Fourier regression](https://otexts.com/fpp3/useful-predictors.html#fourier-series).

<div class="alert alert-block alert-warning">
<b>Simplification.</b> 

There are many academic papers and book chapters about the application of Fourier regression for time series forecasting: the interested reader may refer to [A. Incremona, Machine Learning methods for long and short term energy demand forecasting](https://iris.unipv.it/retrieve/handle/11571/1436355/470615/Incremona_PhD_Thesis.pdf), [A. Incremona, G. De Nicolao, Spectral Characterization of the Multi-Seasonal Component of the Italian Electric Load: A LASSO-FFT Approach](https://ieeexplore.ieee.org/abstract/document/8734852), and [A. Guerini, G. De Nicolao, Long-term electric load forecasting: A torus-based approach](https://ieeexplore.ieee.org/abstract/document/7330957).
</div>

Here, you opt for a simple unregularized linear model:
$$
y(t) = \mu + \kappa t + \sum_{i=1}^n\left[\alpha_i\sin(2\pi\frac{i}{7}t) + \beta_i\cos(2\pi\frac{i}{7}t)\right] + \sum_{j=1}^m\left[\gamma_j\sin(2\pi\frac{j}{365.25}t) + \delta_j\cos(2\pi\frac{j}{365.25}t)\right] + \tau \chi_t
$$
where:
- the terms $\mu + \kappa t$ model a linear trend
- $\chi_t$ is the indicator function taking value 1 when date $t$ is a holiday and 0 otherwise
- the sums
$$
\sum_{i=1}^n\left[\alpha_i\sin(2\pi\frac{i}{7}t) + \beta_i\cos(2\pi\frac{i}{7}t)\right]
$$
and
$$
\sum_{j=1}^m\left[\gamma_i\sin(2\pi\frac{j}{365.25}t) + \delta_j\sin(2\pi\frac{j}{365.25}t)\right]
$$
model the weekly and yearly periodicity

The parameters of the model are $\mu$, $\kappa$, $\{\alpha_i\}$, $\{\beta_i\}$, $\{\gamma_j\}$, $\{\delta_j\}, \tau$, which can all be trained by least squares. 

The hyperparameters are $n$ and $m$, which can be chosen by cross-validation.

In [ ]:
import datetime
import os
import time
from pathlib import Path

import holidays
import numpy as np

## Feature preparation
To perform a Fourier regression, you need proper inputs. Therefore, you build a new dataset.

You add the trend and the holiday dummy, then you build the Fourier harmonics.

<div class="alert alert-block alert-warning">
<b>Simplification.</b> 

In the real world, it would be good practice to choose the number of harmonics with data-driven methods: cross-validation is one, LASSO is another.
Here we just set a fixed value, which we know performs good enough for our purposes.
</div>

In [ ]:
n_harmonics = {
    '7': 5,
    '365': 20
}

dates = load_series.index
regression_df = pd.DataFrame(index=dates)

# create holiday series, valued 0 or 1 depending on the day
holiday_cal = holidays.Italy()
regression_df['holiday'] = dates.to_series().apply(lambda s: s in holiday_cal).astype(int)

# turn dates into integer incremental values to be used as the x axis
date_origin = pd.to_datetime('2006-01-01')
regression_df['trend'] = (dates - date_origin).days

# create fourier terms as trigonometric functions of various frequencies
for i in range(1, n_harmonics['7'] + 1):
    regression_df[f'sin_7_{i}'] = np.sin(2 * np.pi * i / 7 * regression_df['trend'])
    regression_df[f'cos_7_{i}'] = np.cos(2 * np.pi * i / 7 * regression_df['trend'])
for i in range(1, n_harmonics['365'] + 1):
    regression_df[f'sin_365_{i}'] = np.sin(2 * np.pi * i / 365.25 * regression_df['trend'])
    regression_df[f'cos_365_{i}'] = np.cos(2 * np.pi * i / 365.25 * regression_df['trend'])

# join all features together
regression_data_df = resampled_df.join(regression_df)

## Model training

Training the model is often done by using well-known libraries, such as [scikit-learn](https://scikit-learn.org/stable/); this makes everything straightforward, although it arguably hides the inner structure and complexity that ML algorithms have.

In [ ]:
TRAIN_END = '2020-12-31 23:59'

In [ ]:
# split data into train and test
train_df, test_df = regression_data_df[:TRAIN_END], regression_data_df[TRAIN_END:]
train_df.head()

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# split training set into features and target
train_x, train_y = train_df.drop(columns='Load'), train_df['Load']
test_x, test_y = test_df.drop(columns='Load'), test_df['Load']

# instantiate and train scikit-learn's LinearRegression model
model = LinearRegression()
fit = model.fit(train_x, train_y)


In [ ]:
# save the model into a serialized format, ready to be used when needed
import pickle
with open("trained_model.pkl", 'wb') as f:
    pickle.dump(fit, f)

## Model inspection

Let's take a look at the output of the model, that is the parameters that were selected by the training algorithm.

In [ ]:
# extract information on estimated coefficients into a dataframe
model_output_df = pd.DataFrame(index=pd.Index(["intercept"] + list(fit.feature_names_in_), name="component"), data={"coefficient": [fit.intercept_] + list(fit.coef_)})

In [ ]:
from ipywidgets import interact, widgets

fig = px.bar(
    np.abs(model_output_df),
    title = "Absolute value of estimated coefficients",
    log_y=True
       )
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
from datetime import date

selector_widget = widgets.Dropdown(
        description=f'Frequency:   ',
        value=7,
        options=[7, 365]
    )
@interact(frequency=selector_widget, name="Select frequency")
def plot_component(frequency):
  if frequency==7:  # align index so that it starts on monday
    start = 7 - date_origin.weekday()
  else:
    start = 0
  index = np.arange(start, start + frequency)
  component = pd.Series(0, index=index)

  # reconstruct the predicted sum of components having the selected frequency
  for i in range(1, n_harmonics[str(frequency)] + 1):

    # sin component
    sin_coeff = model_output_df.at[f"sin_{frequency}_{i}", "coefficient"]
    harmonic_sin = np.sin(2 * np.pi * i / frequency * index) * sin_coeff
    component += harmonic_sin

    # cos component
    cos_coeff = model_output_df.at[f"cos_{frequency}_{i}", "coefficient"]
    harmonic_cos = np.cos(2 * np.pi * i / frequency * index) * cos_coeff
    component += harmonic_cos

  # set date index for better chart readability
  date_index = pd.date_range(
      date_origin + pd.offsets.DateOffset(days=start),
      date_origin + pd.offsets.DateOffset(days=start + frequency - 1)
      )
  component.index = date_index
  fig = px.line(component,
                title = f"Sum of predicted components with frequency={frequency}",
                width=1100)
  if frequency==7:
    fig.update_layout(xaxis = dict(tickformat = '%A'))
  elif frequency==365:
    fig.update_layout(xaxis = dict(tickformat = '%e %B'))
  fig.update_layout(showlegend=False)
  fig.show()

## Prediction and evaluation
Finally, we use the trained model to perform some predictions and evaluate the results. 
In order to do so, we will use the Mean Absolute Percentage Error:
$$
\mathrm{MAPE(A, F)}=\frac{100 \%}{n} \sum_{t=1}^n\left|\frac{A_t-F_t}{A_t}\right|
$$
where $A$ and $F$ are the actual and forecasted series respectively.

In [ ]:
# apply model on test dates
test_predictions = model.predict(test_x)
test_results_df = pd.DataFrame({"actual": test_df.Load, "predicted": test_predictions})
px.line(test_results_df, title="Predictions on the test set").show()

In [ ]:
# compute MAPE on test predictions
np.mean(np.abs(test_results_df["actual"] - test_results_df["predicted"]) / test_results_df["predicted"])

## Goodness of fit
In order to assess the validity of a model, it is useful to plot actual vs predicted values and see how much they resemble an identity, which is plotted as the bisector of the first quadrant in the following chart.

To help assess this, a line is fit to the scatter data by minimizing the sum of squared distances (OLS).

In [ ]:
fig = px.scatter(test_results_df, x="actual", y="predicted", trendline="ols", title="Goodness of fit", height=800, width=900)
fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.data[-1].marker.color = "black"
fig.data[-1].name = "OLS fit"
fig.data[-1].showlegend = True
fig.add_trace(go.Scatter(x= np.arange(500000, 1000000, 5000), y=np.arange(500000, 1000000, 5000), mode = 'lines', name="actual=predicted"))

fig.show()

Similar information can be delivered by plotting the prediction errors (or residuals) against time in order to check "problematic" dates or periods where the error lies too far from the ideal zero.

In [ ]:
resid_series = test_results_df["actual"] - test_results_df["predicted"]
resid_series.name = "Residual"

In [ ]:
px.line(resid_series, title="Difference between actual and predicted values")

Lastly, the same autocorrelation plot we used at the beginning shows us that the residual carries much less information than the original series in terms of repeating patterns; this means we extracted much of the predictable patterns that were encoded in the data, although we could definitely do something better as low lags still show significant autocorrelation.

In [ ]:
plot_acf(
    resid_series,
    lags=50,
    ).show()

# 5 - Model monitoring
After settling for a satisfactory model, you need to keep monitoring its performance while it's being used by the company.

For this, you set up a rolling measurement (moving average) of the model's performance, which tells you if it is behaving as expected or you need to worry about it, and potentially try better approaches.

In [ ]:
future_df = pd.read_csv("data/2022_data.csv", index_col=0, parse_dates=True).resample('D').sum()

In [ ]:
future_dates = future_df.index
future_regression_df = pd.DataFrame(index=future_dates)

future_regression_df['holiday'] = future_dates.to_series().apply(lambda s: s in holiday_cal).astype(int)

future_regression_df['trend'] = (future_dates - date_origin).days

for i in range(1, n_harmonics['7'] + 1):
    future_regression_df[f'sin_7_{i}'] = np.sin(2 * np.pi * i / 7 * future_regression_df['trend'])
    future_regression_df[f'cos_7_{i}'] = np.cos(2 * np.pi * i / 7 * future_regression_df['trend'])
for i in range(1, n_harmonics['365'] + 1):
    future_regression_df[f'sin_365_{i}'] = np.sin(2 * np.pi * i / 365.25 * future_regression_df['trend'])
    future_regression_df[f'cos_365_{i}'] = np.cos(2 * np.pi * i / 365.25 * future_regression_df['trend'])

In [ ]:
future_predictions = fit.predict(future_regression_df)
future_results_df = pd.DataFrame({"actual": future_df.Load, "predicted": future_predictions})

In [ ]:
window_size = 3
extended_future_results_df = pd.concat([test_results_df[-window_size:], future_results_df], axis=0)
rolling_mape_df = pd.DataFrame(
    {'rolling_mape': map(lambda w: np.mean(np.abs(w["actual"] - w["predicted"]) / w["predicted"]),
                         extended_future_results_df.rolling(window_size))},
    index=extended_future_results_df.index
)
fig = px.line(rolling_mape_df[window_size:], title=f"{window_size} days rolling MAPE in 2022")
fig.show()

In [ ]:
extended_future_results_df